In [ ]:
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization, ZeroPadding2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./100,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./100)

In [ ]:
train_data_dir = 'Data/Heatmaps/Train-Normed-Float-Gauss/TrainGaussianNormalizedCenter/'
validation_data_dir = 'Data/Heatmaps/Train-Normed-Float-Gauss/TrainGaussianNormalizedCenter-Val/'
nb_train_samples = 1117 + 1150 + 1241
nb_validation_samples = 112 + 115 + 124
epochs = 100
batch_size = 16


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(256, 256),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(256, 256),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

In [ ]:
callback = ModelCheckpoint(filepath='Workshop-Paper/Vanilla-Normed/vanila-normed-{epoch:03d}-{val_acc:.3f}', save_best_only=True, monitor='val_acc', verbose=1)

In [ ]:
history = model.fit_generator(train_generator, callbacks=[callback],
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.load_weights('Workshop-Paper/Vanilla-Normed/vanila-normed-063-0.601')

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1.0/100)
test_data_stream = test_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Normed-Float-Gauss/ValidateGaussianNormalizedCenter/',
                    target_size=(256, 256),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

In [ ]:
predictions = model.predict_generator(test_data_stream, 1979, verbose=1)

In [ ]:
actual_classes = test_data_stream.classes

In [ ]:
len(actual_classes)

In [ ]:
pred_classes = np.argmax(predictions, axis=1)

In [ ]:
len(pred_classes)

In [ ]:
print ('Vanila Normed Validation Accuracy Score:', accuracy_score(pred_classes, actual_classes))

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1.0/100)
train_data_stream = train_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Normed-Float-Gauss/TrainGaussianNormalizedCenter/',
                    target_size=(256, 256),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

In [ ]:
predictions = model.predict_generator(train_data_stream, 3157, verbose=1)
actual_classes = train_data_stream.classes
pred_classes = np.argmax(predictions, axis=1)
print ('Vanila Normed Training Accuracy Score:', accuracy_score(pred_classes, actual_classes))

## Training AlexNet on Raw Images

In [ ]:
DROPOUT = 0.5
N_CATEGORY = 3
model_input = Input(shape = (3, 227, 227))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)

In [ ]:
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9
BATCH_SIZE = 128
LEARNING_RATE = 0.01
DROPOUT = 0.5
ALPHA = 1e-4
BETA = 0.75
n = 5
k = 2

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay = WEIGHT_DECAY))

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./100,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./100)

In [ ]:
train_data_dir = 'Data/Train/'
validation_data_dir = 'Data/Train-Val/'
nb_train_samples = 1043 + 1079 + 1145
nb_validation_samples = 116 + 120 + 127
epochs = 100
batch_size = 16

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(227, 227),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(227, 227),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

In [ ]:
callback = ModelCheckpoint(filepath='Workshop-Paper/Alexnet-Raw/vanila-raw-{epoch:03d}-{val_acc:.3f}', save_best_only=True, monitor='val_acc', verbose=1)

In [ ]:
history = model.fit_generator(train_generator, callbacks=[callback],
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )